# Docker Jupyter

https://github.com/comafire/skp_d4r_jupyter

Jupyter는 데이터사이언티스트들이 자주 애용하는 코드에디터 입니다. Kernel 이라는 일종의 모듈 방식을 통해 인터프리터 방식의 다양한 개발 언어를 지원하며, Markdown 지원을 통해 코드에 다양한 설명 및 주석을 추가 할 수 있어 빠르게 데이터 분석, 시각화, 문서화를 하는데 유용합니다.

하지만, 현재까지는 툴에 접속하기 위한 방식이 보안적으로 충분히 안전하지 않기 때문에 로컬 네트워크 상에서 사용하는 것을 추천합니다.

만약, 외부에서 접속하여 사용하고 싶다면 VPN 또는 SSH Forwarding을 이용하고, 로컬 시스템에 의존적 작업을 하지 않는다면 Docker를 통해 로컬 시스템 환경과 격리하는 것을 추천합니다.

위 방법을 사용할 수 없고, 외부로 오픈해야 한다면 최소한 HTTPS 프로토콜을 사용하여 통신 암호화를 하고, SSH로 접속 후 기동 하였다가 작업 후에는 중지 시켜놓는 것을 추천합니다. HTTPS 를 적용하기 위해서는 traefik (https://comafire.github.io/dataplatform/skp_n4e_traefik.html) 을 이용한 방식을 참조하세요.

## 설치

먼저, SKP2를 사용하기 위한 기본 설치를 아래 링크를 통해 진행합니다.

https://comafire.github.io/dataplatform/skp.html

skp_n4e_jupyter 프로젝트를 다운받아 아래 순서대로 설치합니다.

```bash
> cd ~/skp
> git clone https://github.com/comafire/skp_d4r_jupyter.git
> cd skp_d4r_jupyter
```

이제 .envrc.template 파일을 복사해 자신의 환경에 맞게 환경 변수들을 수정해 줍니다.

```bash
# BASE (주의: 환경 변수의 기준 값으로 변수 값 내에 다른 환경 변수를 참조하면 안됨)
export SKP_USER="skp"
export SKP_USERID=$(id -u $SKP_USER)
export SKP_USER_HOME="/home/skp"
export SKP_HOME="/home/skp/skp"

# LOCALE
export LOCALE="ko_KR.UTF-8" # or "en_US.UTF-8" 

# PYTHON
export PYENV_PYVER="3.8.9"
export PYENV_VENV="skp-d4r-jupyter"
export PATH="$HOME/.pyenv/bin:${PATH}"
eval "$(pyenv init --path)"
eval "$(pyenv init -)"
eval "$(pyenv virtualenv-init -)"
export PYENV_VIRTUALENV_DISABLE_PROMPT=1
pyenv activate ${PYENV_VENV} 

# NODEJS - N
export N_PREFIX="${SKP_USER_HOME}/n"; [[ :$PATH: == *":$N_PREFIX/bin:"* ]] || PATH+=":$N_PREFIX/bin"  # Added by n-install (see http://git.io/n-install-repo).

# JUPYTER
export JUPYTER_HOME="${SKP_HOME}/skp_d4r_jupyter"
export JUPYTER_NAME="skp-d4r-jupyter"
export JUPYTER_RUNTIME="" # "" (cpu) or "nvidia" (gpu)
export JUPYTER_IMAGE="skp-d4r-jupyter:latest" # skp-jupyter-sts:latest or skp-jupyter-sts-gpu:latest
export JUPYTER_PORT="52020" # Your Jupyter Port
export JUPYTER_GPU="FALSE" # TRUE or FALSE
export JUPYTER_BASEURL="jupyter" # Your Jupyter BaseURL, ex) http://localhost:8888/jupyter
export JUPYTER_ROOTDIR="${SKP_HOME}/skp_d4r_jupyter_data" # container 안의 notebook dir 에 마운트될 위치
# # HTTPS 적용은 Traefik 에서 진행되므로 자체 SSL 적용은 사용하지 않음
# export JUPYTER_DNS="${LETSENCRYPT_DNS}"
# export JUPYTER_CERT="${LETSENCRYPT_HOME}/var/${JUPYTER_DNS}/cert.pem"
# export JUPYTER_CERT_KEY="${LETSENCRYPT_HOME}/var/${JUPYTER_DNS}/privkey.pem"

# SECRET
. ./.secret
```

다음으로 .secret.template 파일을 .secret 이름으로 복사하고 사용할 암호를 수정해 줍니다. 접근권한을 600 으로 설정하여 다른 사용자가 해당 파일에 접근할 수 없도록 설정합니다.

```bash
# JUPYTER
export JUPYTER_PASSWD="YOUR_PASSWORD" # Your Jupyter Password
```

이제 설정된 환경 변수를 이용하여 jupyter 를 설치 및 기동 할 수 있습니다.

```bash
> direnv allow
> pyenv install $PYENV_PYVER
> pyenv virtualenv $PYENV_PYVER $PYENV_VENV
> pyenv virtualenvs
> direnv allow
> pip install --upgrade pip 
> pip install setuptools invoke wheel

> ./bin/skp.sh -l
Available tasks:

  jupyter.build
  jupyter.restart
  jupyter.start
  jupyter.stop

> ./bin/skp.sh jupyter.build
> ./bin/skp.sh jupyter.start
```